http://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html
https://osf.io/w7tds/  unravelling

http://nbviewer.jupyter.org/github/mathewzilla/redcard/blob/master/Crowdstorming_visualisation.ipynb

Randomforest
https://www.kaggle.com/c/titanic/details/getting-started-with-random-forests



In [1]:
import pandas as pd                                     # for dealing with csv import
import numpy as np                                      # arrays and other matlab like manipulation
import os                                               # for joining paths and filenames sensibly
import matplotlib.pyplot as plt 

In [2]:

df = pd.read_csv('CrowdstormingDataJuly1st.csv')


In [3]:
df.head()

,playerShort,player,club,leagueCountry,birthday,height,weight,position,games,victories,...,rater2,refNum,refCountry,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp
0,lucas-wilchez,Lucas Wilchez,Real Zaragoza,Spain,31.08.1983,177.0,72.0,Attacking Midfielder,1,0,...,0.50,1,1,GRC,0.326391,712.0,0.000564,0.396000,750.0,0.002696
1,john-utaka,John Utaka,Montpellier HSC,France,08.01.1982,179.0,82.0,Right Winger,1,0,...,0.75,2,2,ZMB,0.203375,40.0,0.010875,-0.204082,49.0,0.061504
2,abdon-prats,Abdón Prats,RCD Mallorca,Spain,17.12.1992,181.0,79.0,NaN,1,0,...,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
3,pablo-mari,Pablo Marí,RCD Mallorca,Spain,31.08.1993,191.0,87.0,Center Back,1,1,...,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
4,ruben-pena,Rubén Peña,Real Valladolid,Spain,18.07.1991,172.0,70.0,Right Midfielder,1,1,...,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002


In [4]:
len(df.index)

146028

In [5]:
# Display the first 10 rows of the dataset. Only 13 columns for space reasons
df.ix[:10,:13]

,playerShort,player,club,leagueCountry,birthday,height,weight,position,games,victories,ties,defeats,goals
0,lucas-wilchez,Lucas Wilchez,Real Zaragoza,Spain,31.08.1983,177.0,72.0,Attacking Midfielder,1,0,0,1,0
1,john-utaka,John Utaka,Montpellier HSC,France,08.01.1982,179.0,82.0,Right Winger,1,0,0,1,0
2,abdon-prats,Abdón Prats,RCD Mallorca,Spain,17.12.1992,181.0,79.0,NaN,1,0,1,0,0
3,pablo-mari,Pablo Marí,RCD Mallorca,Spain,31.08.1993,191.0,87.0,Center Back,1,1,0,0,0
4,ruben-pena,Rubén Peña,Real Valladolid,Spain,18.07.1991,172.0,70.0,Right Midfielder,1,1,0,0,0
5,aaron-hughes,Aaron Hughes,Fulham FC,England,08.11.1979,182.0,71.0,Center Back,1,0,0,1,0
6,aleksandar-kolarov,Aleksandar Kolarov,Manchester City,England,10.11.1985,187.0,80.0,Left Fullback,1,1,0,0,0
7,alexander-tettey,Alexander Tettey,Norwich City,England,04.04.1986,180.0,68.0,Defensive Midfielder,1,0,0,1,0
8,anders-lindegaard,Anders Lindegaard,Manchester United,England,13.04.1984,193.0,80.0,Goalkeeper,1,0,1,0,0
9,andreas-beck,Andreas Beck,1899 Hoffenheim,Germany,13.03.1987,180.0,70.0,Right Fullback,1,1,0,0,0


In [6]:
df.columns.values


array(['playerShort', 'player', 'club', 'leagueCountry', 'birthday',
       'height', 'weight', 'position', 'games', 'victories', 'ties',
       'defeats', 'goals', 'yellowCards', 'yellowReds', 'redCards',
       'photoID', 'rater1', 'rater2', 'refNum', 'refCountry', 'Alpha_3',
       'meanIAT', 'nIAT', 'seIAT', 'meanExp', 'nExp', 'seExp'], dtype=object)

In [7]:
df.dtypes

playerShort       object
player            object
club              object
leagueCountry     object
birthday          object
height           float64
weight           float64
position          object
games              int64
victories          int64
ties               int64
defeats            int64
goals              int64
yellowCards        int64
yellowReds         int64
redCards           int64
photoID           object
rater1           float64
rater2           float64
refNum             int64
refCountry         int64
Alpha_3           object
meanIAT          float64
nIAT             float64
seIAT            float64
meanExp          float64
nExp             float64
seExp            float64
dtype: object

# Mean skin tone

We see that there aren't any players with only one skin colour rating, thus it should be alright to find the means of those who have two rating and use those for RandomForest classification.

In [8]:
dfnew = pd.DataFrame(df, columns=['player', 'rater1', 'rater2'])
dfnew.ix[200:300,:]

,player,rater1,rater2
200,Sébastien Grax,0.00,0.25
201,Sébastien Roudet,0.00,0.25
202,Sigamary Diarra,NaN,NaN
203,Sloan Privat,NaN,NaN
204,Steeve Elana,0.75,0.75
205,Stéphane Darbion,NaN,NaN
206,Stéphen Drouin,NaN,NaN
207,Steven N'Zonzi,NaN,NaN
208,Sylvio Rodelin,0.50,0.50
209,Thiago Xavier,NaN,NaN


Generate new column which is the mean of two rater's opinions about the skin colour.

In [9]:
df['skinTone'] = ((df['rater1']+df['rater2'])/2).dropna()

In [10]:
dfnew = pd.DataFrame(df, columns=['player', 'rater1', 'rater2', 'skinTone'])
dfnew.head(10)

,player,rater1,rater2,skinTone
0,Lucas Wilchez,0.25,0.50,0.375
1,John Utaka,0.75,0.75,0.750
2,Abdón Prats,NaN,NaN,NaN
3,Pablo Marí,NaN,NaN,NaN
4,Rubén Peña,NaN,NaN,NaN
5,Aaron Hughes,0.25,0.00,0.125
6,Aleksandar Kolarov,0.00,0.25,0.125
7,Alexander Tettey,1.00,1.00,1.000
8,Anders Lindegaard,0.25,0.25,0.250
9,Andreas Beck,0.00,0.00,0.000


In [11]:
df = df[pd.notnull(df['skinTone'])]
df.head()

,playerShort,player,club,leagueCountry,birthday,height,weight,position,games,victories,...,refNum,refCountry,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp,skinTone
0,lucas-wilchez,Lucas Wilchez,Real Zaragoza,Spain,31.08.1983,177.0,72.0,Attacking Midfielder,1,0,...,1,1,GRC,0.326391,712.0,0.000564,0.396000,750.0,0.002696,0.375
1,john-utaka,John Utaka,Montpellier HSC,France,08.01.1982,179.0,82.0,Right Winger,1,0,...,2,2,ZMB,0.203375,40.0,0.010875,-0.204082,49.0,0.061504,0.750
5,aaron-hughes,Aaron Hughes,Fulham FC,England,08.11.1979,182.0,71.0,Center Back,1,0,...,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752,0.125
6,aleksandar-kolarov,Aleksandar Kolarov,Manchester City,England,10.11.1985,187.0,80.0,Left Fullback,1,1,...,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752,0.125
7,alexander-tettey,Alexander Tettey,Norwich City,England,04.04.1986,180.0,68.0,Defensive Midfielder,1,0,...,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752,1.000


The number of dropped rows is not too large only around 15% of the initial dataset.

In [13]:
len(df.index)

124621

In [14]:
dif = 146028-124621
percent = dif/146028 * 100
percent

14.659517352836444

skinTone = skin rating of photo by rater (5-point scale ranging from “very light skin” to “very dark skin”)
So, we could categorize the skinTone feature, so that over 2.5 it is dark and under it is white.

In [17]:
df['skinTone'].describe()

count    124621.000000
mean          0.283559
std           0.288517
min           0.000000
25%           0.000000
50%           0.250000
75%           0.375000
max           1.000000
Name: skinTone, dtype: float64

As we see, the values are between 0 and 1, median being at 0.25, so there are more whites than people with dark skin. 
Alexander Tettey has a score of 1, and when we Google his name we see that he does have a very dark skin.

# Random Forests

In [18]:
from sklearn.ensemble import RandomForestClassifier

Good example : https://www.kaggle.com/c/titanic/details/getting-started-with-random-forests